In [ ]:
import siibra
assert siibra.__version__ == "2.0.0.a01"
import numpy as np
from nilearn import plotting, image
import re
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import pandas as pd
%matplotlib notebook

# Input: some feature  distribution in MNI space 

We compose an artificial input image by merging some functional maps from the DiFuMo atlas, but this could be anything, e.g. an fMRI activation map. The image is built as a NIfTI, but then casted to a siibra volume so we have a reference space attached and can used it properly in the siibra workflows. Getting the NIfTI object from the siibra volume is cheap and easy via `.fetch()`. 

In [ ]:
difumo64 = siibra.get_map(parcellation="difumo 64", space='mni 152', maptype='statistical')
img = image.smooth_img(image.math_img(
        "np.maximum(np.maximum(im1, im2), im3)", 
        im1=difumo64.extract_regional_map(region="3").get_data(), 
        im2=difumo64.extract_regional_map(region="31").get_data(),
        im3=difumo64.extract_regional_map(region="39").get_data(), 
    ), 10)

# we cast the nifti object to a siibra volume, so it has a space attached and can be used properly
input_provider = siibra.factory.imageprovider_from_nifti(img, space='mni 152')
plotting.plot_glass_brain(input_provider.get_data(), alpha=1, cmap='RdBu')

# Split input volume into cluster components

There are many ways to get components out of a feature map. Here we use siibra to 

- draw random points from the distribution encoded by the input volume, then 
- cluster them using DBSCAN, and 
- build clusterwise featuremaps as Kernel Density estimates thereof.

In this example, this more or less inverts the composition of the input volume from the DiFuMo maps, but the idea for a general input image is to separate it into components that have more meaningful correlations with brain regions than the full image, which is usually a mixture distribution. 

In [ ]:
np.random.seed(seed=27)
N = 10000  # number of random samples
# drawing the samples results in a siibra PointSet, 
# which has reference space attached and can model point uncertainties.
samples = siibra.factory.pointcloud_sampled_from_image(input_provider, N, e=5, sigma=3)

# finding the clusters will result in a labelling of the point set.
clusters = samples.find_clusters(min_fraction=1/300, max_fraction=1/2)
clusterlabels = set(clusters.labels) - {-1}

# Let's have a look at the clustered pointcloud

cmap = cm.rainbow(np.linspace(0, 1, max(clusters.labels) + 1))
view = plotting.plot_glass_brain(input_provider.get_data(), alpha=1, threshold=15, cmap='RdGy')
view.add_markers(
    np.array(clusters.coordinates)[np.array(clusters.labels) >= 0],
    marker_size=5,
    marker_color=[cmap[l] for l in clusters.labels if l >= 0]
)

# Assign peaks and clusters to cytoarchitectonic regions

To assign the clusters to brain regions, we build feature maps from each cluster and assign them to the Julich-Brain probabilistic maps. The assignemint is one-to-many since the structures in the image and parcellation are continuous. Assignments report correlation, intersection over union, and some other measures which we can use to filter and sort them.

The result is an assignment table from cluster components in the input volume to regions in the Julich-Brain atlas.

In [4]:
min_correlation = 0.3
min_map_value = 0.5
pmaps = siibra.get_map(parcellation="julich 3.1", space="mni 152", maptype='statistical', name='207')
assignments = []

In [ ]:
# assign peaks to regions
peaks = siibra.factory.pointcloud_from_image_peaks(input_provider, mindist=5, sigma=0)
df = pmaps.assign(peaks)
df.query(f"`map_value` >= {min_map_value}", engine='python', inplace=True)
df['type'] = 'peak'
df['id'] = df['input_structure_index']   
assignments.append(df[['type', 'id', 'regionname', 'map_value']])

view = plotting.plot_glass_brain(input_provider.get_data(), alpha=1, cmap='RdBu')
view.add_markers(peaks.coordinates, marker_size=30)

In [ ]:
# assign clusters to regions
for l in clusterlabels:
    print(f"cluster {l}")
    clustermap_provider = siibra.factory.imageprovider_from_pointcloud(
        clusters.filter_by_label([l]), target=input_provider
    )
    plotting.plot_glass_brain(
        clustermap_provider.get_data(), alpha=1, cmap="RdBu", title=f"Cluster #{l}"
    )
    df = pmaps.assign(clustermap_provider)
    df.query(f"correlation >= {min_correlation}", engine="python", inplace=True)
    df["type"] = "cluster"
    df["id"] = l
    assignments.append(
        df[["type", "id", "regionname", "correlation", "map_value_mean"]]
    )

In [ ]:
all_assignments = pd.concat(assignments).sort_values(by='map_value_mean', ascending=False)
all_assignments

# Find features

To demonstrate multimodal feateure profiling, we only choose the

In [ ]:
regionname = all_assignments.iloc[0]["regionname"]
pmap = pmaps.extract_regional_map(regionname)
plotting.plot_stat_map(pmap.get_data())

In [ ]:
region = pmaps.get_region(regionname) 
profiling = [
    [siibra.modality_vocab.modality.NEUROTRANSMITTER_RECEPTOR_DENSITY, {}],
    [siibra.modality_vocab.modality.CELL_BODY_DENSITY, {}],
    # [siibra.modality_vocab.modality.STREAMLINECOUNTS, {}],
]

In [ ]:
fig, axs = plt.subplots(len(profiling), 1, figsize=(6.0, 2.5 * len(profiling)))

for i, (modality, kwargs) in enumerate(profiling):
    features = siibra.find_features(region, modality, **kwargs)
    features[0].plot(ax=axs[i])
    
plt.tight_layout()

In [ ]:

region = siibra.get_region('3.0.3', region.name)
features = siibra.find_features(region, siibra.modality_vocab.modality.STREAMLINECOUNTS)

ft = features[0]

def shortname(name):
    return (
        re.sub('\s*\(.*\)\s*|\s*Area\s*', ' ', name)
        .replace('left', 'L')
        .replace("hemisphere", "")
        .replace('right', 'R')
        .strip()
    )


dp = ft.get_datarecipe("name == 'Subject: 000'")
df = dp.get_data()
profile = df[region.name]
sorted_series = profile.sort_values(ascending=False)
named_sorted_series = sorted_series.rename(index={r:shortname(r) for r in sorted_series.index})
named_sorted_series[:15].plot(kind="bar")
